In [1]:
import ee
import geemap
ee.Initialize()

### Get Dynamic World V1 tif image over the ROI
We use landCover from Dynamic World data to determine the area we want to get soil moisture. There are three areas that need to be concerned:

1 . tree

2 . grass

4 . crop

1, 2, 4 is the id in the Dynamic World data.

In [ ]:
import ee
ee.Initialize()

START = '2022-10-01'
END = '2022-10-31'

# Your region of interest
# geojson_path = 'china/map/china_grid.geojson'
# roi = geemap.geojson_to_ee(geojson_path)

# Manually upload the ROI file and load it from Earth Engine assets
roi = ee.FeatureCollection('projects/ee-lengocthanh/assets/1km_china_roi').geometry()

dw = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
    .filterDate(START, END) \
    .filterBounds(roi) \
    .select('label') \
    .mode()

# Export as GeoTIFF
task = ee.batch.Export.image.toDrive(
    image=dw,
    description='dw_china_1km',
    folder='gee_exports',
    fileNamePrefix='dw_china_1km',
    region=roi.bounds(),
    scale=10,
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()


### Filter and retain grid cells meeting the requirements

Using landcover data to filter grids, keep grids that have large are of classes we concered (tree, grass, crop). Then choose random points in filtered grid to get data.

![Filtered Grid](/mnt/data2tb/Transfer-DenseSM-E_pack/images_for_slides/filtered_grids.png)


In [ ]:
import geopandas as gpd
import rasterio
import rasterio.mask
import numpy as np
import pandas as pd

# Choose 'india' or 'china' as your region
region = 'india'

# Load and convert your grid to EPSG:4326
grid = gpd.read_file(f"{region}/grid/2km_grid.gpkg").to_crs("EPSG:4326")

# Open the Dynamic World image (already downloaded as GeoTIFF)
with rasterio.open(f"{region}/grid/dw_{region}_1km.tif") as src:
    results = []
    # Traverse each grid cell in the GeoDataFrame
    for idx, row in grid.iterrows():
        # Get the geometry of the grid cell
        geom = [row['geometry']]
        try:
            # Mask the land cover raster with the grid cell geometry
            out_image, out_transform = rasterio.mask.mask(src, geom, crop=True)
            # Land cover labels are in the first band
            label_pixels = out_image[0]
            # Remove nodata values
            valid_pixels = label_pixels[label_pixels != src.nodata]
            
            if len(valid_pixels) == 0:
                continue
            
            # Count frequencies of each label 
            unique, counts = np.unique(valid_pixels, return_counts=True)
            label_count = dict(zip(unique, counts))

            # Sum up pixels for 3 classes 1 (tree), 2 (grass), 4 (crop)
            selected_total = sum(label_count.get(k, 0) for k in [1,2,4])
            total = sum(counts)

            # Calculate the percentage of selected classes 
            percent = selected_total / total * 100

            # If the percentage is greater than 50%, add the grid cell to results
            if percent > 50:
                results.append(row['id'])  # or use row.name, or any identifier

        except Exception as e:
            print(f"Failed on grid cell {row['id']}: {e}")

# all grid cell ids
all_ids = grid['id']

# Create a new GeoDataFrame with only the selected grid cells (selected based on the results)
selected_grid = grid[grid['id'].isin(results)]

# Save to GPKG
selected_grid.to_file(f"{region}/grid/filtered_grid/tree_grass_crops.gpkg", driver="GPKG")


### Không thể dùng code chung để chọn điểm lấy dữ liệu trên vùng Trung Quốc và Ấn Độ

Bởi vì có sự đặc biệt ở vùng lấy dữ liệu tại Trung Quốc. Ở đó đa số toàn đồi núi do đó, nếu lấy điểm ngẫu nhiên trong các grid cell được lọc sẽ thường rơi vào các vùng là rừng núi. Cách giải quyết đó là, xét trong từng grid cell, nếu tại ô đó có vùng crop sẽ chọn điểm ngẫu nhiên trên crop, còn không có crop trong ô, thì sẽ phải bắt buộc chọn điểm ngẫu nhiên trên vùng rừng núi.

Còn với Ấn Độ, vùng lấy dữ liệu là đồng bằng nên không cần phải làm phức tạp như vậy. Chỉ đơn giản là chọn điểm ngẫu nhiên trong từng grid cell

#### Choose random points for India

In [ ]:
import geopandas as gpd
import shapely
import random
import pandas as pd

# Read the grid from a GPKG file
grid = gpd.read_file(f"india/grid/filtered_grid/tree_grass_crops.gpkg").to_crs("EPSG:4326")

points = []
# Traverse each grid cell in the filtered GeoDataFrame
for idx, row in grid.iterrows():
    polygon = row['geometry']
    grid_id = row['id']
    # Generate a random point within the polygon of the grid cell
    minx, miny, maxx, maxy = polygon.bounds
    while True:
        random_point = shapely.geometry.Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if polygon.contains(random_point):
            break
    points.append({'id': grid_id, 'lon': random_point.x, 'lat': random_point.y})

# Save to CSV
points_df = pd.DataFrame(points)
points_df.to_csv(f"india/grid/filtered_grid/random_points_in_filtered_grid.csv", index=False)

#### Choose random points for China

In [ ]:
import rasterio.mask
import rasterio.transform
import numpy as np
import random
import shapely

# Paths to filtered grid and land cover raster files
grid_path = "china/grid/filtered_grid/tree_grass_crop.gpkg"
dw_tif_path = "china/grid/dw_china_1km.tif"

grid = gpd.read_file(grid_path).to_crs("EPSG:4326")

points = []

# Open land cover raster
with rasterio.open(dw_tif_path) as src:
    # Traverse each grid cell in the filtered GeoDataFrame
    for idx, row in grid.iterrows():
        polygon = row['geometry']
        grid_id = row['id']
        # Get the geometry of the grid cell
        geom = [polygon]

        try: 
            out_image, out_transform = rasterio.mask.mask(src, geom, crop=True)
            label_pixels = out_image[0]
            valid_mask = label_pixels != src.nodata

            # find crop pixels (lable : 4) mask
            crop_mask = (label_pixels == 4) & valid_mask 

            # If there are crop pixels in the grid cell, choose a random one
            if np.any(crop_mask):
                # Get indices of crop_pixels
                crop_indices = np.argwhere(crop_mask)
                # Choose a random crop pixel
                y, x = crop_indices[random.randint(0, len(crop_indices)-1)]
                # convert pixel indices to coordinates 
                lon, lat = rasterio.transform.xy(out_transform, y, x)
            
            # No crop region, pick random point in grid cell
            else:
                minx, miny, maxx, maxy = polygon.bounds 
                while True:
                    random_point = shapely.geometry.Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
                    if polygon.contains(random_point):
                        lon, lat = random_point.x, random_point.y
                        break
            points.append({'id': grid_id, 'lon': lon, 'lat': lat})
        
        except Exception as e:
            print(f"Failed on grid cell {grid_id} : {e}")

# Save to CSV
points_df = pd.DataFrame(points)
points_df.to_csv("china/grid/filtered_grid/random_points_in_filtered_grid.csv", index=False)           
